In [ ]:
%load_ext autoreload
%autoreload 2shange

In [ ]:
# import osmnx as oximport pandas as pd
import geopandas as gpd
import mytools
import math
import numpy as np
from shapely.ops import unary_union
import pandas as pd

In [ ]:
from shapely.geometry import Polygon,Point,LineString,MultiPolygon
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import matplotlib.pyplot as plt
import simplekml

In [ ]:
#导入竞对数据
jingd = pd.read_csv('G:/1-规划/评估/软采评估/2020年9月/3-输出内容/各地市软采小区数据/软采小区-武汉.csv',encoding='gbk')
gongcan = pd.read_csv('G:/1-规划/评估/软采评估/2020年9月/1-基础数据/基站数据库/ltejizhan20200903.csv',encoding='gbk',usecols=['小区CGI(*)','RRU经度', 'RRU纬度'])

In [ ]:
#导入投诉数据
tousu = pd.ExcelFile('G:/1-规划/1其他工作输出/20200925四个区县进行top平估/基础数据/广义投诉CI表-0910_TO赵曼(1).xlsx')

# 导入区域底图图层

In [ ]:
city = '区分公司名称'
quadrat_width=1000
city_name = '青山'

In [ ]:
#导入栅格数据
f = mytools.othern.file_name_paths('G:/1-规划/评估/软采评估/2020年二季度/3-输出内容/栅格数据/武汉/')
wuhan_grid = pd.DataFrame()
for name in f:
    if  city_name in ['青山','洪山','武昌','东新']:
        data_t = pd.read_csv(name,encoding='gbk')
        wuhan_grid = wuhan_grid.append(data_t)

In [ ]:
quxian = mytools.gisn.maps('武汉')
quxian

In [ ]:
qingshan = quxian.loc[quxian[city]==city_name]

In [ ]:
qingshan = mytools.gisn.buffer(qingshan,0.001)

In [ ]:
qingshan.plot(figsize =(13,13),color= '#BFBFBF',edgecolor='#000000', lw=0.4)#,k = 4, cmap = plt.cm.Greens

# 使用线条划分底图为栅格

In [ ]:
#转化坐标系
qingshan = qingshan.to_crs({'init': 'epsg:32650'})
#去除geometry
geometry = qingshan['geometry'].iloc[0]
if isinstance(geometry, Polygon):
    geometry = MultiPolygon([geometry])
#用线划分geometry生成Mutipolygon
geometry_cut = mytools.gisn.quadrat_cut_geometry(geometry, quadrat_width=quadrat_width)
#替换geometry
qingshan['geometry'].iloc[0] = geometry_cut
#拆分MultiPolygon
qingshan_grid = qingshan.explode()
#转回坐标系
qingshan_grid = qingshan_grid.to_crs({'init': 'epsg:4326'})
#保留部分列
qingshan_res = qingshan_grid.reset_index()[[city,'geometry']]
#添加id编号
qingshan_res['id'] = [ 'region_grid_{}'.format(str(name)) for name in qingshan_res.index+1]
#导出tab格式数据
# qingshan_res.to_file('d:/qingshan.TAB',driver="MapInfo File",encoding='gbk')

In [ ]:
qingshan_res.plot(figsize =(13,13),color= '#BFBFBF',edgecolor='#000000', lw=0.4)

In [ ]:
qingshan_res.shape

# 导入栅格数据

In [ ]:
wuhan_grid.head(1)

In [ ]:
# wuhan_grid['弱覆盖栅格数量']=np.nan
# wuhan_grid.loc[(wuhan_grid['有效栅格']==1) & (wuhan_grid['fgl']<0.936),'弱覆盖栅格数量']=1

# wuhan_grid['严重弱覆盖栅格数量']=np.nan
# wuhan_grid.loc[(wuhan_grid['有效栅格']==1) & (wuhan_grid['fgl']<0.7),'严重弱覆盖栅格数量']=1

# qingshan_grid_data = wuhan_grid.loc[wuhan_grid['区县']=='青山']

In [ ]:
qingshan_data_p = mytools.gisn.add_points(wuhan_grid,'lon','lat')

# 区域栅格和软采栅格数据匹配

In [ ]:
res= gpd.sjoin(qingshan_res,qingshan_data_p)

In [ ]:
res.columns

In [ ]:
res_groupby = res.groupby('id').agg({'a.rsrp_sample_count':'sum','a.rsrp_weak':'sum','a.grid_id':'count','有效栅格':'count','弱覆盖栅格数量':'count','严重弱覆盖栅格数量':'count'})

In [ ]:
res_groupby = res_groupby.reset_index()

In [ ]:
res_groupby.head(1)

In [ ]:
res_groupby.shape

# 区域栅格和投诉量匹配

In [ ]:
tousu.sheet_names

In [ ]:
tousu_1_4 = tousu.parse()

In [ ]:
tousu_1_4.columns

In [ ]:
tousu_1_4.head(1)

In [ ]:
tousu_1_4 = tousu_1_4.loc[(tousu_1_4['经度']>0) & (tousu_1_4['纬度']>0)]

In [ ]:
tousu_1_4['经度'] = pd.to_numeric(tousu_1_4['经度'])

In [ ]:
tousu_1_4['纬度'] = pd.to_numeric(tousu_1_4['纬度'])

In [ ]:
tousu_1_4['2020年投诉量'] = tousu_1_4['2020年6月投诉量']+tousu_1_4['2020年7月投诉量']+tousu_1_4['2020年8月投诉量']

In [ ]:
tousu_p = mytools.gisn.add_points(tousu_1_4,'经度','纬度')

In [ ]:
res_tousu= gpd.sjoin(qingshan_res,tousu_p)

In [ ]:
tousu_groupby = res_tousu.groupby('id').agg({'2020年投诉量':'sum'})

In [ ]:
tousu_groupby = tousu_groupby.reset_index()

In [ ]:
qingshan_res2 = res_groupby.merge(tousu_groupby,how='left',on='id')

加入竞对的数据

In [ ]:
gongcan_use = gongcan.loc[gongcan['RRU经度']>0]

In [ ]:
jingd_use = jingd.merge(gongcan_use,how='left',on='小区CGI(*)')

In [ ]:
jingd_p = mytools.gisn.add_points(jingd_use.loc[jingd_use['RRU经度']>0],'RRU经度','RRU纬度')

In [ ]:
res_tousu2= gpd.sjoin(qingshan_res,jingd_p)

In [ ]:
tousu_groupby2 = res_tousu2.groupby('id').agg({'hd':'sum'})

In [ ]:
tousu_groupby2 = tousu_groupby2.reset_index()

In [ ]:
qingshan_res2.head(1)

In [ ]:
tousu_groupby2.head(1)

In [ ]:
qingshan_res2 = qingshan_res2.merge(tousu_groupby2,how='left',on='id')

# 结果放入区域栅格列中

In [ ]:
qingshan_res2.sort_values('hd',ascending=False)

In [ ]:
qingshan_res3 = qingshan_res.merge(qingshan_res2,how='left',on='id')

In [ ]:
qingshan_res3.head(1)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax = qingshan_res3.plot(ax=ax, column='a.rsrp_sample_count', cmap =plt.cm.Greens, edgecolor='#000000', lw=0.2)

#关闭坐标轴
# ax.axis('off')
# plt.show()

# 评分系统

In [ ]:
qingshan_res3.head(1)

In [ ]:
qingshan_res4 = qingshan_res3[['id','a.rsrp_sample_count','a.rsrp_weak','a.grid_id','有效栅格','弱覆盖栅格数量','严重弱覆盖栅格数量','2020年投诉量','hd','geometry']]

In [ ]:
qingshan_res4.columns=['id', '总点', '弱点', '栅格数', '有效栅格', '弱覆盖栅格数量','严重弱覆盖栅格数量','投诉量','黑点','geometry']

In [ ]:
qingshan_res4['覆盖率'] = 1 - qingshan_res4['弱点']/qingshan_res4['总点']

In [ ]:
qingshan_res4['总点'].max()

In [ ]:
qingshan_res4['k1']=20 * (qingshan_res4['弱点']-1000)/(qingshan_res4['弱点'].max()-1000)
qingshan_res4.loc[(qingshan_res4['弱点']<1000),'k1']=0

In [ ]:
qingshan_res4['k2']=30 * (qingshan_res4['严重弱覆盖栅格数量'])/(qingshan_res4['严重弱覆盖栅格数量'].max())

In [ ]:
qingshan_res4['k3']=qingshan_res4['覆盖率'].apply(lambda x:(1-(max(min(x,0.936),0.7)-0.7)/(0.936-0.7))*20)
qingshan_res4.loc[(qingshan_res4['覆盖率']<0.7),'k3']=20
qingshan_res4.loc[(qingshan_res4['覆盖率']>=0.936),'k3']=0

In [ ]:
qingshan_res4['k4']=20 * (qingshan_res4['投诉量'])/(qingshan_res4['投诉量'].max())

In [ ]:
qingshan_res4['k5']=10 * (qingshan_res4['黑点'])/(qingshan_res4['黑点'].max())

In [ ]:
qingshan_res4['k']=qingshan_res4['k1']+qingshan_res4['k2']+qingshan_res4['k3']+qingshan_res4['k4']+qingshan_res4['k5']

In [ ]:
qingshan_res4 = qingshan_res4.sort_values('k',ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax = qingshan_res4.plot(ax=ax, column='k', cmap =plt.cm.Reds, edgecolor='#000000', lw=0.2)


In [ ]:
qingshan_res4['经纬度'] = qingshan_res4.centroid

# qingshan_res4.to_clipboard()

# 选取top

In [ ]:
# data = pd.read_clipboard()

# qingshan_res5 = qingshan_res4.merge(data,how='right',on='id')

In [ ]:
qingshan_res4.info()

In [ ]:
shuchu_g = pd.read_csv('G:/1-规划/1其他工作输出/20200925四个区县进行top平估/需要输出图层的栅格.csv',encoding='gbk')
zong_g = pd.read_csv('G:/1-规划/1其他工作输出/20200925四个区县进行top平估/四个区县的总得分.csv',encoding='gbk')

In [ ]:
qingshan_res4 = qingshan_res4.drop(columns='经纬度')

In [ ]:
qingshan_res4 = qingshan_res4.dropna(subset=['总点', '弱点'])

In [ ]:
qingshan_res4['总点弱点'] =qingshan_res4.apply(lambda x:str(int(x['总点']))+"_"+str(int(x['弱点'])),axis=1)

In [ ]:
qingshan_res4 = qingshan_res4.reset_index(drop=True)

In [ ]:
qingshan_res4.columns

In [ ]:
qingshan_res4_2 = qingshan_res4[['总点弱点','geometry']]

In [ ]:
zong_g.columns

In [ ]:
zong_g = zong_g.astype('object')

In [ ]:
qingshan_res4_2.columns=['count_sum','geometry']

In [ ]:
qingshan_res4_2['count_sum'] = qingshan_res4_2['count_sum'].astype('object')

In [ ]:
qingshan_res4_2.to_csv('G:/1-规划/1其他工作输出/20200925四个区县进行top平估/linshi2.csv',encoding='gbk',index=False)

In [ ]:
qingshan_res4_3 = pd.read_csv('G:/1-规划/1其他工作输出/20200925四个区县进行top平估/linshi3.csv',encoding='gbk')

In [ ]:
qingshan_res4 = mytools.gisn.read_geocsv(qingshan_res4_3)

In [ ]:
qingshan_res4.shape

In [ ]:
qingshan_res4[['lon','lat']] = qingshan_res4.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [ ]:
qingshan_res4.to_clipboard()

In [ ]:
data_linshi = pd.read_csv('G:/1-规划/1其他工作输出/20200925四个区县进行top平估/linshi.csv',encoding='gbk')

In [ ]:
qingshan_res4 = mytools.gisn.read_geocsv()

In [ ]:
# qingshan_res5 = qingshan_res4.head(5)

In [ ]:
qingshan_res5.to_csv(f'G:/1-规划/1其他工作输出/20200925四个区县进行top平估/{city_name}top5.csv',encoding='gbk',index=False)

# 制作图层

In [ ]:
qingshan_res5 = qingshan_res4.copy()

In [ ]:
qingshan_res5 = qingshan_res5.reset_index(drop=True)

In [ ]:
# qingshan_res5 = qingshan_res5.drop(columns='经纬度')

In [ ]:
qingshan_res5.columns

In [ ]:
qingshan_res5.columns = ['count_sum', 'geometry','id', 'rsrp_count', 'rsrp_weak', 'grid_count', 'grid_100_count', 'grid_weak', 'grid_weak_Serious', 'complain','hd',
       'fgl', 'k1', 'k2', 'k3', 'k4', 'k5', 'k']

In [ ]:
#制作tab图层f
qingshan_res5.to_file(f'G:/1-规划/1其他工作输出/20200925四个区县进行top平估/{city_name}top5.tab',driver="MapInfo File",encoding='gbk')

In [ ]:
# qingshan_res5 = gpd.read_file('d:/qingshan.tab',encoding='gbk')

In [ ]:
# qingshan_res5

In [ ]:
#制作google图层

In [ ]:
qingshan_res6 = qingshan_res5.reset_index()

In [ ]:
qingshan_res6['lin'] = qingshan_res6.boundary

In [ ]:
kml = simplekml.Kml()

In [ ]:
def to_kml(x):
    lin = kml.newlinestring(
        name=x['id'],                   
        description=x[[ 'id','rsrp_count','rsrp_weak','grid_count','grid_100_count','grid_weak','grid_weak_Serious','complain','fgl','k']].to_string(),
        coords=list(x['lin'].coords))
    lin.style.linestyle.color =simplekml.Color.changealphaint(245, 'ff00a5ff' )
    lin.style.linestyle.width= 5

In [ ]:
city_name = '洪山'

In [ ]:
qingshan_res6_2 = qingshan_res6.loc[qingshan_res6['id'].str.contains(city_name)]

In [ ]:
kml = simplekml.Kml()
qingshan_res6_2.apply(to_kml,axis=1)
kml.save(f'G:/1-规划/1其他工作输出/20200925四个区县进行top平估/{city_name}top.kml')
print('ok')

In [ ]:
mytools.dog()

In [ ]:
res_tousu2_r = res_tousu2[['区分公司名称',  'id',  '小区CGI(*)', '地市', '区县',
       'rsrp_sample_count', 'rsrp_weak', '区域类型', 'a.un_maxrsrp_sample_count',
       'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count',
       'a.te_maxrsrp_weak_l2', 'fgl_cmcc', 'fgl_un', 'fgl_te',
       'te_hd', 'un_hd', 'hd', 'RRU经度', 'RRU纬度']]

In [ ]:
res_tousu2_r.to_csv(f'G:/1-规划/1其他工作输出/20200925四个区县进行top平估/{city_name}top5_竞对.csv')

In [ ]:
res_tousu_r = res_tousu[['区分公司名称', 'eNodeB名称', '经度', '纬度',
       'eNodeBid', '地市', '区县', '站点类型', '2020年6月投诉量', '2020年7月投诉量',
       '2020年8月投诉量', '2020年投诉量']]

In [ ]:
res_tousu_r.columns = ['区分公司名称', 'eNodeB名称', '经度', '纬度', 'eNodeBid', '地市', '区县', '站点类型',
       '2020年6月投诉量', '2020年7月投诉量', '2020年8月投诉量', '投诉量总计']

In [ ]:
res_tousu.to_csv(f'G:/1-规划/1其他工作输出/20200925四个区县进行top平估/{city_name}top5_投诉.csv')

In [ ]:
qingshan_res4

In [ ]:
res_tousu2= gpd.sjoin(qingshan_res4,jingd_p)

In [ ]:
res_tousu= gpd.sjoin(qingshan_res4,tousu_p)

In [ ]:
res_shange= gpd.sjoin(qingshan_res4,qingshan_data_p)

In [ ]:
res_tousu2.to_clipboard()

In [ ]:
res_tousu.to_clipboard()

In [ ]:
res_shange.to_clipboard()

In [ ]:
quxian1 = mytools.gisn.maps('地市区县')

In [ ]:
quxian1.columns

In [ ]:
quxian1['lin'] = quxian1.boundary

In [ ]:
def to_kml2(x):
    lin = kml.newlinestring(
        name=x['区县'],                   
        description=x[[ '地市','区县']].to_string(),
        coords=list(x['geometry'].exterior.coords))
    lin.style.linestyle.color =simplekml.Color.changealphaint(245, 'ffffffff' )
    lin.style.linestyle.width= 2

In [ ]:
quxian1.apply(to_kml2,axis=1)
kml.save('G:/1-规划/图层/县城和地市归属图层pickle/地市区县.kml')
print('ok')

In [ ]:
gongcan = pd.read_csv('g:/1-规划/工参/工参数据/ltejizhan20200807.csv',encoding='gbk')

In [ ]:
gongcan.columns

In [ ]:
a = set(gongcan['归属区县'])

In [ ]:
f=open("d:/k.txt","w")
f.write(str(a))
f.close()

In [ ]:
mytools.gisn.points_coverage_merge()

# top30匹配周边的7734、催开、故障、优化、高负荷、干扰

In [ ]:
qingshan_res5.shape

In [ ]:
#导入7734汇总的数据

In [ ]:
data_7734 = pd.ExcelFile('G:/1-规划/1其他工作输出/20200603青山按照大栅格划分问题点/3-分析top解决方案/1基础数据/青山专属建设、优化、故障/汇总/建设、故障、优化汇总.xlsx')

In [ ]:
data_7734.sheet_names

In [ ]:
data_7734.sheet_names

In [ ]:
data_7734_sum = data_7734.parse(sheet_name='汇总')

In [ ]:
data_7734_sum.head(1)

In [ ]:
data_7734_sum.loc[data_7734_sum['区县']=='青山区'].shape

In [ ]:
p_7734 = mytools.gisn.add_points(data_7734_sum)

In [ ]:
top30_7734_sjoin = gpd.sjoin(qingshan_res5,p_7734)

In [ ]:
top30_7734_sjoin.columns

In [ ]:
top30_7734_sjoin.drop_duplicates('id').shape

In [ ]:
top30_7734_sjoin.drop_duplicates('唯一编号').shape

In [ ]:
top30_7734_sjoin.drop_duplicates('唯一编号').shape[0]/data_7734_sum.loc[data_7734_sum['区县']=='青山区'].shape[0]

In [ ]:
top30_7734_sjoin.drop_duplicates('唯一编号').groupby('分类').count()

# 匹配现网小区

In [ ]:
gongcan= pd.read_csv('G:/1-规划/工参/工参数据/ltejizhan20200529.csv',encoding='gbk',usecols=['小区CGI(*)', 'RRU经度', 'RRU纬度','覆盖场景'])

In [ ]:
gongcan.columns

In [ ]:
changjing_shuoming = pd.read_csv('G:/1-规划/表格类保存/场景说明.csv',encoding='gbk')

In [ ]:
changjing_shuoming.head(1)

In [ ]:
gongcan_use = gongcan.merge(changjing_shuoming,how='left',left_on='覆盖场景',right_on='类型ID')

In [ ]:
gogncan_p = mytools.gisn.points_coverage_merge(gongcan_use.loc[gongcan_use['RRU经度']>0],'RRU经度', 'RRU纬度')

In [ ]:
gogncan_p2 = mytools.gisn.add_points(gogncan_p, 'RRU经度', 'RRU纬度')

In [ ]:
gogncan_p2 = mytools.gisn.add_points(gogncan_p, 'RRU经度', 'RRU纬度')

In [ ]:
top30_gongcan_sjoin = gpd.sjoin(qingshan_res5,gogncan_p2)

In [ ]:
top30_gongcan_sjoin.columns

In [ ]:
top30_gongcan_sjoin.head(1)

In [ ]:
res_changjing = top30_gongcan_sjoin.drop_duplicates('小区CGI(*)').groupby([ '场景类型']).agg({'id':'count'})

In [ ]:
res_changjing.to_clipboard()

# 匹配软采负荷干扰

In [ ]:
#软采小区处理

In [ ]:
ruancai = pd.read_csv('G:/1-规划/1其他工作输出/20200603青山按照大栅格划分问题点/3-分析top解决方案/1基础数据/覆盖/汇总-按照采样点排序去重.csv',
    encoding='gbk')

In [ ]:
ruancai_use = ruancai[[ 'eutrancell_cgi', 'rsrp_sample_count', 'rsrp_weak', 'RRU经度', 'RRU纬度']]

In [ ]:
top30_gongcan_sjoin.groupby([ '区域类型']).agg({'id':'count'})

In [ ]:
ruancai_use['fgl'] = 1 - ruancai_use['rsrp_weak']/ruancai_use['rsrp_sample_count']

In [ ]:
ruancai_use = ruancai_use.loc[ruancai_use['fgl']<0.9]

In [ ]:
ruancai_p = mytools.gisn.add_points(ruancai_use, 'RRU经度', 'RRU纬度')

In [ ]:
#干扰小区处理

In [ ]:
ganrao = pd.read_csv('G:/1-规划/1其他工作输出/20200603青山按照大栅格划分问题点/3-分析top解决方案/1基础数据/干扰/干扰小区汇总.csv',
    encoding='gbk')

In [ ]:
ganrao_use = ganrao.merge(gongcan,how='left',left_on='小区ID',right_on='小区CGI(*)').drop_duplicates('小区ID')[['小区ID','RRU经度','RRU纬度']]

In [ ]:
ganrao_p = mytools.gisn.add_points(ganrao_use,'RRU经度','RRU纬度')

In [ ]:
ganrao.head(1)

In [ ]:
#负荷小区处理

In [ ]:
fuhe = pd.read_csv('G:/1-规划/1其他工作输出/20200603青山按照大栅格划分问题点/3-分析top解决方案/1基础数据/高负荷/高负荷汇总.csv',
    encoding='gbk')

In [ ]:
fuhe.head(1)

In [ ]:
fuhe_use = fuhe.merge(gongcan,how='left',left_on='CGI',right_on='小区CGI(*)').drop_duplicates('CGI')[['CGI','RRU经度','RRU纬度']]

In [ ]:
fuhe_p = mytools.gisn.add_points(fuhe_use,'RRU经度','RRU纬度')

In [ ]:
#匹配到大栅格

In [ ]:
rancai_top_sjoin = gpd.sjoin(qingshan_res5,ruancai_p)

In [ ]:
ganrao_top_sjoin = gpd.sjoin(qingshan_res5,ganrao_p)

In [ ]:
fuhe_top_sjoin = gpd.sjoin(qingshan_res5,fuhe_p)

In [ ]:
#计算结果

In [ ]:
#软采

In [ ]:
rancai_top_sjoin.to_clipboard()

In [ ]:
rancai_top_sjoin.drop_duplicates('eutrancell_cgi').shape

In [ ]:
#干扰

In [ ]:
ganrao_top_sjoin.to_clipboard()

In [ ]:
ganrao_top_sjoin.drop_duplicates('小区ID').shape

In [ ]:
#负荷

In [ ]:
fuhe_top_sjoin.to_clipboard()

In [ ]:
fuhe_top_sjoin.drop_duplicates('CGI').shape

In [ ]:
#现网

In [ ]:
top30_gongcan_sjoin.drop_duplicates('小区CGI(*)').shape

In [ ]:
#写word用

In [ ]:
id = 'region_grid_138'

In [ ]:
qingshan_res5.columns

In [ ]:
b=qingshan_res5.loc[qingshan_res5['id']==id][['总点','覆盖率','弱覆盖栅格数量','严重弱覆盖栅格数量','投诉量']].reset_index()

In [ ]:
xq=str(top30_gongcan_sjoin.loc[top30_gongcan_sjoin['id']==id].agg({'小区CGI(*)':'count'})['小区CGI(*)'])

In [ ]:
'该区域主要弱覆盖包括***以及周边楼宇。区域内共计{}个小区，整体采样点{}个相对较多，覆盖率为{}， \
区域内有{}个栅格覆盖率小于93.6%，{}个栅格覆盖率低于70%，20年投诉共计{}个。 \
'.format(xq,int(b['总点'][0]),str(b['覆盖率'][0]*100)[:5]+'%',int(b['弱覆盖栅格数量'][0]),int(b['严重弱覆盖栅格数量'][0]),int(b['投诉量'][0]))

In [ ]:
'区域内'+xq+ \
'个现网小区中，有'+str(rancai_top_sjoin.loc[rancai_top_sjoin['id']==id].agg({'eutrancell_cgi':'count'})['eutrancell_cgi'])+ \
'个弱覆盖小区，'+str(ganrao_top_sjoin.loc[ganrao_top_sjoin['id']==id].agg({'小区ID':'count'})['小区ID'])+ \
'个为干扰小区，'+str(fuhe_top_sjoin.loc[fuhe_top_sjoin['id']==id].agg({'CGI':'count'})['CGI'])+'个为高负荷小区'#负荷

In [ ]:
a=top30_7734_sjoin.loc[top30_7734_sjoin['id']==id].groupby('分类').agg({'唯一编号':'count'})

In [ ]:
a

In [ ]:
'区域内与7734工单重合{}单，其中可新建站点{}个，可优化小区{}个，可进行维护处理站点{}个。 \
可根据实际情况进行推动工单处理进度。改善区域的网络质量。'.format(a.sum()[0],a.loc['建设'][0],a.loc['优化'][0],a.loc['故障'][0])